In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/Thesis/dataset/data_conll

/content/drive/.shortcut-targets-by-id/1W5BzjqZFipKZwnOFzfFiVT0UJTt7Ej8F/Thesis/dataset/data_conll


Dependency che cả 3

In [ ]:
import csv
import xml.etree.ElementTree as ET
import os
import spacy
def extract_data(array_text, array_pr):
    # Iterate through each word in the array
    predicat = ""
    global predicat_indx
    for index, word in enumerate(array_text):
        # Remove the '#' if it exists
        if word.startswith('#'):
            word = word[1:]
            array_text[index] = array_text[index][1:]
            predicat = word
            predicat_indx = index
        # Add the word to the sentence
        # If the word is not '.', add a space after it
    # Print the resulting sentence
    return predicat, array_text, array_pr

def mask_abolished_tokens(predicat, doc, first_words, third_words):
    masked_words = []
    masked_third_words = []
    global noHead1
    global noHead2
    i = 0
    noHead1 = 0
    noHead2 = 0
    for token in doc:
        print(token.text, "--", token.dep_, "-->", token.head.text)
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A0' in third_words[i]:
                first_words_temp = first_words.copy()
                first_words_temp[i] = ''
                masked_words.append(first_words_temp)
                print('add 0')
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead1+=1
    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A1' in third_words[i]:
                print(i, "-->",token.head.text,  '-->', token.text, '-->', first_words[i], '-->', third_words[i])

                first_words_temp_1 = first_words.copy()
                first_words_temp_1[i] = ''
                masked_words.append(first_words_temp_1)
                print(masked_words)
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead2+=1
    return masked_words, masked_third_words

def process_file(records, array_data, array_pr):
    global lines
    nlp = spacy.load("en_core_web_sm")
    for index, line_array in enumerate(array_data):

        if line_array:
            predicat, first_words, third_words = extract_data(line_array, array_pr[index])

            test = ' '.join(first_words[:-1]) + ('.' if first_words[-1] == '.' else ' ' + first_words[-1])
            print(test)
            doc = nlp(test)
            masked_words, masked_third_words = mask_abolished_tokens(predicat, doc, first_words, third_words)


            data_1, data_2 = generate_output_data(predicat, first_words, masked_words, masked_third_words, records[index], third_words)
            print(data_1)
            print(data_2)


def generate_output_data(predicat, first_words, masked_words, masked_third_words, record, third_words):
    data_1 = ""
    data_2 = ""
    masked_word_pr = first_words.copy()
    masked_third_word_pr = third_words.copy()
    del masked_third_word_pr[predicat_indx]
    del masked_word_pr[predicat_indx]
    with open('./progress/ner_coNLL_testa_abolish.txt', 'a') as file:
        print(noHead1)
        for i, words in enumerate(masked_words):
            for j, word in enumerate(words):
                if word == predicat:
                    masked_words[i][j] = "#" + predicat
        for i, word in enumerate(first_words):
            if word == predicat:
                first_words[i] = "#" + predicat
        masked_words_1 = [[word for word in words if word != ''] for words in masked_words]
        print(len(masked_words_1) - 1)
        masked_third_words_1 = [[word for word in words if word != ''] for words in masked_third_words]
        for i in range(len(masked_words_1)):
            print(i)
            file.write(str(record) + "    " + str(masked_third_words_1[i]) + "    " + str(masked_words_1[i]) + "\n")
            with open('./progress/ner_coNLL_testa_abolish_dup.txt', 'a') as dupFil:
                dupFil.write( str(record) + "	" + str(third_words) + "	" +str(first_words) + "\n")
                dupFil.close()
            with open('./progress/ner_coNLL_testa_abolish_pr.txt', 'a') as dupFil:
                dupFil.write( str(record) + "	" + str(masked_third_word_pr) + "	" +str(masked_word_pr) + "\n")
                dupFil.close()
    return data_1, data_2
# Open the TSV file
with open('ner_coNLL_srl_dev_abolish.tsv', 'r', newline='') as file:
    # Create a CSV reader object with tab delimiter
    reader = csv.reader(file, delimiter='\t')

    # Initialize an empty list to store array data
    array_data = []

    array_pr = []
    records = []
    global predicat

    # Iterate through each row in the file
    for row in reader:
        # Extract the array data from the second column and convert it into a list
        array = eval(row[1])
        array_text = eval(row[2])
        record = eval(row[0])
        # Append the array data to the list
        array_data.append(array_text)
        array_pr.append(array)
        records.append(record)
    process_file(records, array_data, array_pr)
# Print the array data

the a/t-rich mut sequence indicates that normal splicing was abolished by a g-to-a transition at the first nucleotide of intron 2.
the -- det --> sequence
a -- det --> sequence
/ -- punct --> rich
t -- npadvmod --> rich
- -- punct --> rich
rich -- amod --> sequence
mut -- compound --> sequence
sequence -- nsubj --> indicates
indicates -- ROOT --> indicates
that -- mark --> abolished
normal -- amod --> splicing
splicing -- nsubjpass --> abolished
was -- auxpass --> abolished
abolished -- ccomp --> indicates
by -- agent --> abolished
a -- det --> transition
g -- nmod --> transition
- -- punct --> g
to -- prep --> g
- -- punct --> to
a -- pobj --> to
transition -- pobj --> by
at -- prep --> abolished
add 0
the -- det --> nucleotide
first -- amod --> nucleotide
nucleotide -- pobj --> at
of -- prep --> nucleotide
intron -- pobj --> of
2 -- nummod --> intron
. -- punct --> indicates
7 --> abolished --> splicing --> splicing --> I-A1
[['the', 'a/t-rich', 'mut', 'sequence', 'indicates', 'that'

In [ ]:
import csv
import spacy
import os

def extract_data(array_text, array_pr):
    predicat = ""
    predicat_indx = -1
    for index, word in enumerate(array_text):
        if word.startswith('#'):
            word = word[1:]
            array_text[index] = array_text[index][1:]
            predicat = word
            predicat_indx = index
    return predicat, array_text, array_pr, predicat_indx

def mask_abolished_tokens(predicat, doc, first_words, third_words):
    masked_words = []
    masked_third_words = []
    noHead1 = 0
    noHead2 = 0
    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A0' in third_words[i]:
                first_words_temp = first_words.copy()
                first_words_temp[i] = ''
                masked_words.append(first_words_temp)
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead1 += 1

    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A1' in third_words[i]:
                first_words_temp_1 = first_words.copy()
                first_words_temp_1[i] = ''
                masked_words.append(first_words_temp_1)
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead2 += 1

    return masked_words, masked_third_words, noHead1, noHead2

def generate_output_data(predicat, first_words, masked_words, masked_third_words, record, third_words, predicat_indx, noHead1, base_filename):
    masked_word_pr = first_words.copy()
    masked_third_word_pr = third_words.copy()
    del masked_third_word_pr[predicat_indx]
    del masked_word_pr[predicat_indx]

    base_path = './progress/'

    with open(f'{base_path}{base_filename}_depen.tsv', 'a') as file:
        for i, words in enumerate(masked_words):
            for j, word in enumerate(words):
                if word == predicat:
                    masked_words[i][j] = "#" + predicat
        for i, word in enumerate(first_words):
            if word == predicat:
                first_words[i] = "#" + predicat
        masked_words_1 = [[word for word in words if word != ''] for words in masked_words]
        masked_third_words_1 = [[word for word in words if word != ''] for words in masked_third_words]
        for i in range(len(masked_words_1)):
            file.write(f"{record}\t{masked_third_words_1[i]}\t{masked_words_1[i]}\n")
            with open(f'{base_path}{base_filename}.tsv', 'a') as dup_file:
                dup_file.write(f"{record}\t{third_words}\t{first_words}\n")

            with open(f'{base_path}{base_filename}_predicate.tsv', 'a') as pr_file:
                pr_file.write(f"{record}\t{masked_third_word_pr}\t{masked_word_pr}\n")

def process_file(file_path):
    nlp = spacy.load("en_core_web_sm")
    base_filename = os.path.splitext(os.path.basename(file_path))[0]

    with open(file_path, 'r') as file:
        lines = file.readlines()
        array_data = []
        array_pr = []
        records = []
        record_index = 0

        current_words = []
        current_tags = []

        for line in lines:
            if line.strip() == "":
                if current_words and current_tags:
                    array_data.append(current_words)
                    array_pr.append(current_tags)
                    records.append(record_index)
                    current_words = []
                    current_tags = []
                    record_index += 1
            else:
                parts = line.strip().split()
                if len(parts) == 3:
                    word = parts[0]
                    tag = parts[2]
                    current_words.append(word)
                    current_tags.append(tag)

        if current_words and current_tags:
            array_data.append(current_words)
            array_pr.append(current_tags)
            records.append(record_index)

        for index, line_array in enumerate(array_data):
            predicat, first_words, third_words, predicat_indx = extract_data(line_array, array_pr[index])

            test = ' '.join(first_words[:-1]) + ('.' if first_words[-1] == '.' else ' ' + first_words[-1])
            doc = nlp(test)
            masked_words, masked_third_words, noHead1, noHead2 = mask_abolished_tokens(predicat, doc, first_words, third_words)
            generate_output_data(predicat, first_words, masked_words, masked_third_words, records[index], third_words, predicat_indx, noHead1, base_filename)

def main():
    input_dir = './'
    progress_dir = './progress/'
    if not os.path.exists(progress_dir):
        os.makedirs(progress_dir)

    for filename in os.listdir(input_dir):
        if filename.startswith('coNLL_srl_dev'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path)

if __name__ == "__main__":
    main()


In [ ]:
nlp = spacy.load("en_core_web_sm")
test = 'disruption of rpos , mlra or csga , and the curli subunit gene being able to abolish ha and curli production by strain chi7122 interrupt this open reading frame.'
print(test)
doc = nlp(test)
for token in doc:
        print(token.text, "--", token.dep_, "-->", token.head.text)

disruption of rpos , mlra or csga , and the curli subunit gene being able to abolish ha and curli production by strain chi7122 interrupt this open reading frame.
disruption -- nsubj --> interrupt
of -- prep --> disruption
rpos -- pobj --> of
, -- punct --> rpos
mlra -- conj --> rpos
or -- cc --> mlra
csga -- conj --> mlra
, -- punct --> disruption
and -- cc --> disruption
the -- det --> gene
curli -- compound --> subunit
subunit -- compound --> gene
gene -- nsubj --> being
being -- conj --> disruption
able -- acomp --> being
to -- aux --> abolish
abolish -- xcomp --> able
ha -- punct --> abolish
and -- cc --> abolish
curli -- compound --> production
production -- dobj --> abolish
by -- prep --> abolish
strain -- compound --> chi7122
chi7122 -- pobj --> by
interrupt -- ROOT --> interrupt
this -- det --> frame
open -- amod --> frame
reading -- compound --> frame
frame -- dobj --> interrupt
. -- punct --> interrupt


## Constituency che depen

In [27]:
import csv
import xml.etree.ElementTree as ET
import os
import spacy
def extract_data(array_text, array_pr):
    # Iterate through each word in the array
    predicate = ""
    global predicat_indx
    predicat_indx = -1
    for index, word in enumerate(array_text):
        # Remove the '#' if it exists
        if word.startswith('#'):
            word = word[1:]
            array_text[index] = array_text[index][1:]
            predicate = word
            predicat_indx = index
        # Add the word to the sentence
        # If the word is not '.', add a space after it
    # Print the resulting sentence
    return predicate, array_text, array_pr

def mask_abolished_tokens(predicate, doc, first_words, third_words):
    masked_words = []
    masked_third_words = []
    global noHead1
    global noHead2
    i = 0
    noHead1 = 0
    noHead2 = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicate:
            if (token.text in first_words[i]) and 'A0' in third_words[i]:
                first_words_temp = first_words.copy()
                first_words_temp[i] = ''
                masked_words.append(first_words_temp)
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead1+=1
    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicate:
            if (token.text in first_words[i]) and 'A1' in third_words[i]:
                first_words_temp_1 = first_words.copy()
                first_words_temp_1[i] = ''
                masked_words.append(first_words_temp_1)
                temp = third_words.copy()
                temp[i] = ''
                masked_third_words.append(temp)
                noHead2+=1
    return masked_words, masked_third_words


def generate_output_data(predicat, first_words, masked_words, masked_third_words, record, third_words, base_filename):
    data_1 = ""
    data_2 = ""
    masked_word_pr = first_words.copy()
    masked_third_word_pr = third_words.copy()
    if (predicat_indx >= 0):
      del masked_third_word_pr[predicat_indx]
      del masked_word_pr[predicat_indx]
      base_path = './progress_cons/'
      for i, word in enumerate(first_words):
              if word == predicat:
                  first_words[i] = "#" + predicat
      with open(f'{base_path}{base_filename}.tsv', 'a') as filet:
              filet.write(f"{record}\t{third_words}\t{first_words}\n")
      with open(f'{base_path}{base_filename}_predicate.tsv', 'a') as file:
          for i, words in enumerate(masked_words):
              for j, word in enumerate(words):
                  if word == predicat:
                      masked_words[i][j] = ""

          for i, word in enumerate(first_words):
              if word == predicat:
                  first_words[i] = ""
          masked_words_1 = [[word for word in words if word != '' or word == "#" + predicat] for words in masked_words]
          masked_third_words_1 = [[word for word in words if word != ''] for words in masked_third_words]
          file.write(f"{record}\t{masked_third_word_pr}\t{masked_word_pr}\n")

      return data_1, data_2
# Open the TSV file

def process_file(file_path):
    nlp = spacy.load("en_core_web_sm")
    base_filename = os.path.splitext(os.path.basename(file_path))[0]

    with open(file_path, 'r') as file:
        lines = file.readlines()
        array_data = []
        array_pr = []
        records = []
        record_index = 0

        current_words = []
        current_tags = []

        for line in lines:
            if line.strip() == "":
                if current_words and current_tags:
                    array_data.append(current_words)
                    array_pr.append(current_tags)
                    records.append(record_index)
                    current_words = []
                    current_tags = []
                    record_index += 1
            else:
                parts = line.strip().split()
                if len(parts) == 3:
                    word = parts[0]
                    tag = parts[2]
                    current_words.append(word)
                    current_tags.append(tag)

        if current_words and current_tags:
            array_data.append(current_words)
            array_pr.append(current_tags)
            records.append(record_index)

        for index, line_array in enumerate(array_data):
            predicate, first_words, third_words = extract_data(line_array, array_pr[index])

            test = ' '.join(first_words[:-1]) + ('.' if first_words[-1] == '.' else ' ' + first_words[-1])
            doc = nlp(test)
            masked_words, masked_third_words = mask_abolished_tokens(predicate, doc, first_words, third_words)
            generate_output_data(predicate, first_words, masked_words, masked_third_words, records[index], third_words, base_filename)

def main():
    input_dir = './'
    progress_dir = './progress/'
    if not os.path.exists(progress_dir):
        os.makedirs(progress_dir)

    for filename in os.listdir(input_dir):
        if filename.startswith('coNLL_srl_dev'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path)

if __name__ == "__main__":
    main()
        # Print the array data

In [25]:
import os

progress_dir = './progress_cons/'

for filename in os.listdir(progress_dir):
    if filename.startswith('coNLL_srl_dev') and '_predicate' not in filename and '_cons' not in filename:
        file_path = os.path.join(progress_dir, filename)
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"The file {file_path} has been deleted.")

The file ./progress_cons/coNLL_srl_dev_begin.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_alter.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_develop.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_block.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_confer.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_catalyse.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_transform.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_encode.tsv has been deleted.


In [26]:
import os

progress_dir = './progress_cons/'

for filename in os.listdir(progress_dir):
    if filename.startswith('coNLL_srl_dev') and '_predicate' in filename:
        file_path = os.path.join(progress_dir, filename)
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"The file {file_path} has been deleted.")

The file ./progress_cons/coNLL_srl_dev_begin_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_alter_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_develop_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_block_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_confer_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_catalyse_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_transform_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_encode_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_modify_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_recognize_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_transcribe_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_result_predicate.tsv has been deleted.
The file ./progress_cons/coNLL_srl_dev_lose_predicate.

## Constituency che casc tu con lai

In [5]:
!pip install spacy

In [10]:
import csv
import xml.etree.ElementTree as ET
import os
import spacy
def extract_data(array_text, array_pr):
    # Iterate through each word in the array
    predicat = ""
    global predicat_indx
    for index, word in enumerate(array_text):
        # Remove the '#' if it exists
        if word.startswith('#'):
            word = word[1:]
            array_text[index] = array_text[index][1:]
            predicat = word
            predicat_indx = index
        # Add the word to the sentence
        # If the word is not '.', add a space after it
    # Print the resulting sentence
    return predicat, array_text, array_pr

def mask_abolished_tokens(predicat, doc, first_words, third_words):
    masked_words = []
    masked_third_words = []
    global noHead1
    global noHead2
    i = 0
    noHead1 = 0
    noHead2 = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A0' in third_words[i]:
                first_words_temp = first_words.copy()
                temp = third_words.copy()
                for j, words in enumerate(first_words_temp):
                    if words != first_words[i] and 'A0' in third_words[j]:
                        first_words_temp[j] = ''
                        temp[j] = ''
                masked_words.append(first_words_temp)
                masked_third_words.append(temp)
                noHead1+=1
    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A1' in third_words[i]:
                first_words_temp = first_words.copy()
                temp = third_words.copy()
                for j, words in enumerate(first_words_temp):
                    if words != first_words[i] and 'A1' in third_words[j]:
                        first_words_temp[j] = ''
                        temp[j] = ''
                masked_words.append(first_words_temp)
                masked_third_words.append(temp)
                noHead2+=1
    return masked_words, masked_third_words

def generate_output_data(predicat, first_words, masked_words, masked_third_words, record, third_words, base_filename):
    data_1 = ""
    data_2 = ""
    masked_word_pr = first_words.copy()
    masked_third_word_pr = third_words.copy()
    base_path = './progress_cons/'
    with open(f'{base_path}{base_filename}_cons.tsv', 'a') as file:
        for i, word in enumerate(first_words):
            if word == predicat:
                first_words[i] = "#" + predicat
        for i, words in enumerate(masked_words):
            for j, word in enumerate(words):
                if word == predicat:
                    masked_words[i][j] = "#" + predicat
        for i, words in enumerate(masked_words):
            for j, word in enumerate(words):
                if masked_words[i][j] == "":
                    for z in range(len(first_words)):
                        if ('A0' in third_words[i] and 'A0' in third_words[z]):
                          masked_word_pr = first_words.copy()  # Convert strings to list of characters
                          masked_third_word_pr = third_words.copy()
                          masked_word_pr[z] = ''
                          masked_third_word_pr[z] = ''
                          masked_words_1 = [word for word in masked_word_pr if word != '']
                          masked_third_words_1 = [word for word in masked_third_word_pr if word != '']
                          file.write(f"{record}\t{masked_third_words_1}\t{masked_words_1}\n")
                        if ('A1' in third_words[i] and 'A1' in third_words[z]):
                          masked_word_pr = first_words.copy()  #a Convert strings to list of characters
                          masked_third_word_pr = third_words.copy()
                          masked_word_pr[z] = ''
                          masked_third_word_pr[z] = ''
                          masked_words_1 = [word for word in masked_word_pr if word != '']
                          masked_third_words_1 = [word for word in masked_third_word_pr if word != '']
                          file.write(f"{record}\t{masked_third_words_1}\t{masked_words_1}\n")
        # masked_words_1 = [[word for word in words if word != ''] for words in masked_words]
        # masked_third_words_1 = [[word for word in words if word != ''] for words in masked_third_words]
        # for i in range(len(masked_words_1)):
        #     file.write(str(record) + "    " + str(masked_third_words_1[i]) + "    " + str(masked_words_1[i]) + "\n")
    return data_1, data_2

# Open the TSV file
def process_file(file_path):
    nlp = spacy.load("en_core_web_sm")
    base_filename = os.path.splitext(os.path.basename(file_path))[0]

    with open(file_path, 'r') as file:
        lines = file.readlines()
        array_data = []
        array_pr = []
        records = []
        record_index = 0

        current_words = []
        current_tags = []

        for line in lines:
            if line.strip() == "":
                if current_words and current_tags:
                    array_data.append(current_words)
                    array_pr.append(current_tags)
                    records.append(record_index)
                    current_words = []
                    current_tags = []
                    record_index += 1
            else:
                parts = line.strip().split()
                if len(parts) == 3:
                    word = parts[0]
                    tag = parts[2]
                    current_words.append(word)
                    current_tags.append(tag)

        if current_words and current_tags:
            array_data.append(current_words)
            array_pr.append(current_tags)
            records.append(record_index)

        for index, line_array in enumerate(array_data):
            predicate, first_words, third_words = extract_data(line_array, array_pr[index])

            test = ' '.join(first_words[:-1]) + ('.' if first_words[-1] == '.' else ' ' + first_words[-1])
            doc = nlp(test)
            masked_words, masked_third_words = mask_abolished_tokens(predicate, doc, first_words, third_words)
            generate_output_data(predicate, first_words, masked_words, masked_third_words, records[index], third_words, base_filename)

def main():
    input_dir = './'
    progress_dir = './progress/'
    if not os.path.exists(progress_dir):
        os.makedirs(progress_dir)

    for filename in os.listdir(input_dir):
        if filename.startswith('coNLL_srl_dev'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path)

if __name__ == "__main__":
    main()

## Che tưng từ

## Dependency che

In [ ]:
import csv
import xml.etree.ElementTree as ET
import os
import spacy
def extract_data(array_text, array_pr):
    # Iterate through each word in the array
    predicat = ""
    global predicat_indx
    for index, word in enumerate(array_text):
        # Remove the '#' if it exists
        if word.startswith('#'):
            word = word[1:]
            array_text[index] = array_text[index][1:]
            predicat = word
            predicat_indx = index
        # Add the word to the sentence
        # If the word is not '.', add a space after it
    # Print the resulting sentence
    return predicat, array_text, array_pr

def mask_abolished_tokens(predicat, doc, first_words, third_words):
    masked_words = []
    masked_third_words = []
    global noHead1
    global noHead2
    i = 0
    noHead1 = 0
    noHead2 = 0
    for token in doc:
        print(token.text, "--", token.dep_, "-->", token.head.text)
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A0' in third_words[i]:
                first_words_temp = first_words.copy()

                for j, words in enumerate(first_words_temp):
                    if words != first_words[i] and 'A0' in third_words[j]:
                        first_words_temp_1 = first_words.copy()
                        first_words_temp_1[j] = ''
                        temp = third_words.copy()
                        temp[j] = ''
                        masked_words.append(first_words_temp_1)
                        print('add 0')

                        masked_third_words.append(temp)
                noHead1+=1
    i = 0
    for token in doc:
        if token.text not in first_words[i]:
            i += 1
        if token.head.text == predicat:
            if (token.text in first_words[i]) and 'A1' in third_words[i]:
                first_words_temp = first_words.copy()
                for j, words in enumerate(first_words_temp):
                    if words != first_words[i] and 'A1' in third_words[j]:
                        first_words_temp_1 = first_words.copy()
                        first_words_temp_1[j] = ''
                        temp = third_words.copy()
                        temp[j] = ''
                        masked_words.append(first_words_temp_1)
                        print('add 0')

                        masked_third_words.append(temp)
                noHead2+=1
    return masked_words, masked_third_words

def process_file(records, array_data, array_pr):
    global lines
    nlp = spacy.load("en_core_web_sm")
    for index, line_array in enumerate(array_data):

        if line_array:
            predicat, first_words, third_words = extract_data(line_array, array_pr[index])

            test = ' '.join(first_words[:-1]) + ('.' if first_words[-1] == '.' else ' ' + first_words[-1])
            print(test)
            doc = nlp(test)
            masked_words, masked_third_words = mask_abolished_tokens(predicat, doc, first_words, third_words)


            data_1, data_2 = generate_output_data(predicat, first_words, masked_words, masked_third_words, records[index], third_words)
            print(data_1)
            print(data_2)


def generate_output_data(predicat, first_words, masked_words, masked_third_words, record, third_words):
    data_1 = ""
    data_2 = ""
    masked_word_pr = first_words.copy()
    masked_third_word_pr = third_words.copy()
    del masked_third_word_pr[predicat_indx]
    del masked_word_pr[predicat_indx]
    with open('./progress-cons/ner_coNLL_srl_dev_abolish_cons_.tsv', 'a') as file:
        print(noHead1)
        for i, words in enumerate(masked_words):
            for j, word in enumerate(words):
                if word == predicat:
                    masked_words[i][j] = "#" + predicat
        for i, word in enumerate(first_words):
            if word == predicat:
                first_words[i] = "#" + predicat
        masked_words_1 = [[word for word in words if word != ''] for words in masked_words]
        print(len(masked_words_1) - 1)
        masked_third_words_1 = [[word for word in words if word != ''] for words in masked_third_words]
        for i in range(len(masked_words_1)):
            print(i)
            file.write(str(record) + "    " + str(masked_third_words_1[i]) + "    " + str(masked_words_1[i]) + "\n")
    return data_1, data_2
# Open the TSV file
with open('ner_coNLL_srl_dev_abolish.tsv', 'r', newline='') as file:
    # Create a CSV reader object with tab delimiter
    reader = csv.reader(file, delimiter='\t')

    # Initialize an empty list to store array data
    array_data = []

    array_pr = []
    records = []
    global predicat

    # Iterate through each row in the file
    for row in reader:
        # Extract the array data from the second column and convert it into a list
        array = eval(row[1])
        array_text = eval(row[2])
        record = eval(row[0])
        # Append the array data to the list
        array_data.append(array_text)
        array_pr.append(array)
        records.append(record)
    process_file(records, array_data, array_pr)
# Print the array data

the a/t-rich mut sequence indicates that normal splicing was abolished by a g-to-a transition at the first nucleotide of intron 2.
the -- det --> sequence
a -- det --> sequence
/ -- punct --> rich
t -- npadvmod --> rich
- -- punct --> rich
rich -- amod --> sequence
mut -- compound --> sequence
sequence -- nsubj --> indicates
indicates -- ROOT --> indicates
that -- mark --> abolished
normal -- amod --> splicing
splicing -- nsubjpass --> abolished
was -- auxpass --> abolished
abolished -- ccomp --> indicates
by -- agent --> abolished
a -- det --> transition
g -- nmod --> transition
- -- punct --> g
to -- prep --> g
- -- punct --> to
a -- pobj --> to
transition -- pobj --> by
at -- prep --> abolished
add 0
add 0
add 0
add 0
add 0
add 0
add 0
add 0
add 0
the -- det --> nucleotide
first -- amod --> nucleotide
nucleotide -- pobj --> at
of -- prep --> nucleotide
intron -- pobj --> of
2 -- nummod --> intron
. -- punct --> indicates
add 0
1
9
0
1
2
3
4
5
6
7
8
9


disruption of rpos , mlra or c